# Extract DNA/RNA sequence from PDF document

Experimental method to extract DNA/RNA from PDF documents. Use with extreme caution: always validate the extracted sequence!

## Donate! ☕️💸🍻
If this saves you time/money, consider buying me a coffee: 

(BTC) 168QyFEEeYLNNJewdqWkYAZf6Yur1V5k7L

(ETH) 0x9504207811248D1EE8833B090Dfb6494619C3A39

Thank you!

## License
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
!pip install pdfquery

# https://github.com/jcushman/pdfquery
import pdfquery

# Download files from e.g. https://www.bio.purdue.edu/people/faculty/gelvin/nsf/protocols_vectors.htm
f_pdf = 'pSAT6-EYFP-N1_(pE3225)_seq.pdf'

pdf = pdfquery.PDFQuery(f_pdf)
assert pdf.doc.is_extractable

print('Load takes extremely long (up to 15 minutes)')
print('This is due to some inefficient pattern matching in the library')
pdf.load()

In [ ]:
from lxml import etree
import sys
from IPython.core.display import display, HTML

# Inspect what a line of DNA code looks like in the pdf (xml)
root = pdf.pq(':contains("TCAC")')[2]
innerxml = etree.tostring(root, pretty_print=True)
html = innerxml.decode(sys.stdout.encoding)

print("PDF XML:")
display(HTML(html))
print("PDF XML:")
display(html)

In [ ]:
root = pdf.tree
file_xml = etree.tostring(root, pretty_print=True)
# print (file_xml)

In [ ]:
# Find xml parent of sequence:
#  in the documents tested so far, the DNA text was inside LTTextLineHorizontal, 
#  which has the property word_margin. Other LTTextLineHorizontal elements that 
#  did not have DNA has the index attribute specified.

items = root.findall(".//LTTextLineHorizontal[@word_margin]")
err_items = []
# sequence contains two arrays (strands)
sequence = [[], []]
output = ''
line = 1
for item in items:
    # Specify conditions to exclude elements that do not contain DNA/RNA.
    if 'index' not in item.attrib:
        # Validatate that our text is indeed the genetic code
        seq = item.text.strip()
        no_seq = seq.replace('T', '').replace('C', '').replace('A', '').replace('G', '').strip()
        if len(no_seq) > 0:
            err_items.append(item)
        elif len(seq) > 0:
            sequence_len = sum([len("".join(x.split())) for x in sequence[(line + 1) % 2]])
            # Add sequence line to one of the strands
            sequence[(line + 1) % 2].append(seq)
            output += str(sequence_len + 1) + '\t' + seq + '\n'
            if line % 2 == 0: output += '\n'
            line += 1

assert len(sequence[0]) == len(sequence[1]), 'sequence strands are not of same length, something is missing!'

print (len(err_items), 'remarks (non-sequence text) found.')
print('WARN: COPYING THE SEQUENCE DIRECTLY WILL NOT WORK!!')
print('Check if this sequence is correct (length, end, etc) then copy from the *next* cell to Benchling\n')
print(output)
print('WARN: DO NOT COPY SEQUENCE DIRECTLY!! Check the length and copy from the *next* cell to Benchling\n')

**Copy (single strand) output in this cell to Benchling:**

In [ ]:
print("".join(sequence[0]))

### Errors/remarks found during parsing:

In [ ]:
for i in err_items:
    print(i.text)